# Imports

In [1]:
import time
import torch
torch.__version__

'2.5.1'

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available, and using", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("CUDA is not available, using CPU only.")

CUDA is available, and using NVIDIA GeForce GTX 1660 Ti


# Autograd for Ex01

setup

In [3]:
x = torch.tensor(3.0, requires_grad=True) # requires_grad=True enables autograd for tracking computation graph
x

tensor(3., requires_grad=True)

In [4]:
y = x ** 2
y # its has remembered that it came from x via PowBackward0

tensor(9., grad_fn=<PowBackward0>)

calculating grad

In [5]:
y.backward() # computes dy/dx
x.grad # prints the computed gradient dy/dx = 2x = 6

tensor(6.)

# Autograd for Ex02

setup

In [6]:
x = torch.tensor(3.0, requires_grad=True) # requires_grad=True enables autograd for tracking computation graph
x

tensor(3., requires_grad=True)

In [7]:
y = x ** 2
y

tensor(9., grad_fn=<PowBackward0>)

In [8]:
z = torch.sin(y)
z

tensor(0.4121, grad_fn=<SinBackward0>)

calculating grad

In [9]:
z.backward() # computes dz/dx
x.grad # prints the computed gradient dz/dx = cos(y) * dy/dx = cos(x^2) * 2x

tensor(-5.4668)

# Autograd for Ex03

setup

In [10]:
x = torch.tensor(6.7) # here no autograd, since we are tracking weights not x
y = torch.tensor(0.0) # here no autograd, since we are tracking weights not y

x, y

(tensor(6.7000), tensor(0.))

In [11]:
w = torch.tensor(1.0, requires_grad=True) # weights with tracking
b = torch.tensor(0.0, requires_grad=True) # bias with tracking

w, b

(tensor(1., requires_grad=True), tensor(0., requires_grad=True))

In [12]:
z = (w * x) + b
z

tensor(6.7000, grad_fn=<AddBackward0>)

In [13]:
y_pred = torch.sigmoid(z)
y_pred

tensor(0.9988, grad_fn=<SigmoidBackward0>)

In [14]:
def binary_cross_entropy(y_pred, y):
    epsilon = 1e-8 # to prevent log(0) case
    y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)
    result = - (y * torch.log(y_pred)) - ((1 - y) * torch.log(1 - y_pred))
    return result

loss = binary_cross_entropy(y_pred, y)
loss

tensor(6.7012, grad_fn=<SubBackward0>)

calculating grad

In [15]:
loss.backward() # compute gradients of loss w.r.t. w and b
w.grad, b.grad # print the gradients

(tensor(6.6918), tensor(0.9988))

# Autograd Options

clearing grad

In [44]:
x = torch.tensor(3.0, requires_grad=True)
y = x ** 2
x, y

(tensor(3., requires_grad=True), tensor(9., grad_fn=<PowBackward0>))

In [48]:
# y.backward()
# x.grad # if we run backward again, gradients will be accumulated

# To clear the gradients before running backward again
# y.backward()
# x.grad.zero_()
# x.grad

disable tracking

In [51]:
# option 01
# x.requires_grad_(False) # disable tracking

# option 02
# z = x.detach() # creating new graph without tracking

# option 03
# with torch.no_grad():
#     y = x ** 2
#     y.backward()
#     x.grad